In [2]:
#preprocesamiento datos de aprehendidos/detenidos
import pandas as pd
import numpy as np

#CARGA DE DATOS
nombre_archivo = 'detenidosaprehendidos/dataset/mdi_detenidosaprehendidos_pm_2025_enero_octubre.xlsx'

try:
    print("Cargando archivo Excel... esto puede tardar unos segundos...")
    df_apre = pd.read_excel(
        nombre_archivo,
        sheet_name=1,
        dtype={'codigo_parroquia': str},
        engine='openpyxl'
    )
    print(f"Registros totales: {len(df_apre)}")

except Exception as e:
    print("Error cargando el Excel:", e)


Cargando archivo Excel... esto puede tardar unos segundos...
Registros totales: 61149


In [3]:
# Estandarizar código de parroquia
if "codigo_parroquia" in df_apre.columns:
    df_apre["codigo_parroquia"] = (
        df_apre["codigo_parroquia"]
        .astype(str)
        .str.replace(r"\.0$", "", regex=True)
        .str.zfill(6)
    )

# LIMPIEZA DE COORDENADAS
for col in ["latitud", "longitud"]:
    df_apre[col] = (
        df_apre[col]
        .astype(str)
        .str.replace(",", ".", regex=False)
    )
    df_apre[col] = pd.to_numeric(df_apre[col], errors="coerce")

# Eliminar coordenadas inválidas
df_apre = df_apre[
    df_apre[["latitud", "longitud"]].notna().all(axis=1)
    & (df_apre["latitud"] != 0)
    & (df_apre["longitud"] != 0)
].copy()


In [4]:
# PROCESAR FECHA Y HORA
try:
    df_apre["fecha"] = pd.to_datetime(df_apre["fecha_detencion_aprehension"], errors="coerce")

    df_apre["hora_limpia"] = (
        df_apre["hora_detencion_aprehension"]
        .astype(str)
        .str.replace(" ", "")
        .str.strip()
    )

    df_apre["fecha_completa"] = pd.to_datetime(
        df_apre["fecha"].astype(str) + " " + df_apre["hora_limpia"],
        errors="coerce"
    )

except Exception as e:
    print("Advertencia procesando fecha/hora:", e)


In [5]:
# Seleccionar columnas de interés
cols_interes = [
    "fecha_completa", "fecha", "latitud", "longitud",
    "codigo_parroquia", "nombre_parroquia",
    "presunta_infraccion", "tipo", "arma", "movilizacion"
]

df_apre_clean = df_apre[cols_interes].copy()

# Crear características de tiempo
df_apre_clean["franja_horaria"] = df_apre_clean["fecha_completa"].dt.hour
df_apre_clean["dia"] = df_apre_clean["fecha_completa"].dt.day
df_apre_clean["mes"] = df_apre_clean["fecha_completa"].dt.month
df_apre_clean["dia_semana"] = df_apre_clean["fecha_completa"].dt.dayofweek


In [6]:
#GRID ESPACIAL, PARA TENER UN RANGO DE COORDENADAS
df_apre_clean["lat_grid"] = df_apre_clean["latitud"].round(3)
df_apre_clean["lon_grid"] = df_apre_clean["longitud"].round(3)

#Conteo de delito por grid, target
grouped = (
    df_apre_clean
    .groupby(["lat_grid", "lon_grid", "fecha"])
    .size()
    .reset_index(name="conteo_delitos")
)

df_apre_clean = df_apre_clean.merge(
    grouped,
    on=["lat_grid", "lon_grid", "fecha"],
    how="left"
)

In [7]:
print("\n=== Dataset final limpio ===")
print(df_apre_clean.head())


=== Dataset final limpio ===
       fecha_completa      fecha   latitud   longitud codigo_parroquia  \
0 2025-02-03 08:00:00 2025-02-03 -3.715507 -79.618370           071150   
1 2025-05-16 13:40:00 2025-05-16 -1.046660 -77.742967           150155   
2 2025-05-20 13:30:00 2025-05-20 -2.260877 -79.878447           090150   
3 2025-04-11 17:00:00 2025-04-11 -3.270166 -79.953237           070150   
4 2025-09-18 15:30:00 2025-09-18 -0.951131 -79.358865           121150   

     nombre_parroquia                                presunta_infraccion  \
0           PORTOVELO                                            BOLETAS   
1   PUERTO MISAHUALLI                DELITOS CONTRA LOS RECURSOS MINEROS   
2  GUAYAQUIL-FLORESTA                                            BOLETAS   
3             MACHALA  DELITOS POR LA PRODUCCIÓN O TRÁFICO ILÍCITO DE...   
4            VALENCIA  DELITOS POR LA PRODUCCIÓN O TRÁFICO ILÍCITO DE...   

          tipo     arma  movilizacion  franja_horaria  dia  mes  dia